In [1]:
import math
import warnings
import numpy as np
import torch.optim
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
from torchsummary import summary
import random as r
import copy
from BFdataMaker import *
import time
# 示例用法
from scipy.fftpack import fft, ifft
from scipy import signal

In [ ]:
# Attention 层
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, input_dim):
        super(Attention, self).__init__()
        self.W = nn.Linear(input_dim, 1, bias=False)

    def forward(self, input1, input2):
        # 计算注意力权重
        attention_scores = self.W(input1 + input2)
        attention_weights = F.softmax(attention_scores, dim=1)

        # 加权融合特征
        output = attention_weights * input1 + (1 - attention_weights) * input2

        return output

# 假设input1和input2是两类输入数据
input1 = torch.randn(10, 50)  # 10个样本，每个样本50维特征
input2 = torch.randn(10, 50)

# 创建注意力模块
attention = Attention(input_dim=50)

# 融合特征
output = attention(input1, input2)
print(output)

In [2]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model_time = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=5,kernel_size=9,padding=4), 
            nn.BatchNorm2d(5,1e-6),
            nn.ReLU(),
            nn.Conv2d(5,5,kernel_size=2,stride=2), # (40,40)
            nn.BatchNorm2d(5,1e-6),
            nn.ReLU(),
        )
        self.model_corp = nn.Sequential(
            nn.Conv2d(in_channels=5,out_channels=10,kernel_size=7,padding = 3),
            nn.BatchNorm2d(10,1e-6),
            nn.ReLU(),
            nn.Conv2d(10,10,kernel_size=2,stride=2),
            nn.BatchNorm2d(10,1e-6),
            nn.ReLU(),
            nn.Conv2d(in_channels=10,out_channels=15,kernel_size=5,padding = 2),
            nn.BatchNorm2d(15,1e-6),
            nn.ReLU(),
            nn.Conv2d(15,15,kernel_size=2,stride=2),
            nn.BatchNorm2d(15,1e-6),
            nn.ReLU(),
            nn.Conv2d(in_channels=15,out_channels=30,kernel_size=3,padding = 1),
            nn.BatchNorm2d(30,1e-6),
            nn.ReLU(),
            nn.Conv2d(30,30,kernel_size=2,stride=2),
            nn.Flatten(),
            # nn.Linear(750,750),
            # nn.ReLU(),
            nn.Linear(750,3),
            nn.Softmax(dim = 1)
        )
        self.model_fre = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=5,kernel_size=6), 
            nn.BatchNorm2d(5,1e-6),
            nn.ReLU(),
            nn.Conv2d(in_channels=5,out_channels=5,kernel_size=6), 
            nn.BatchNorm2d(5,1e-6),
            nn.ReLU(),
        )
      

    def forward(self, input_tot):
        # print(input,input.shape)
        input_time = input_tot[:,:6400]
        # input_fre = input_tot[:,6400:]
        input_time = input_time.reshape(-1,1,80,80) 
        input_time = self.model_time(input_time)

        # input_fre = input_fre.reshape(-1,1,50,50)
        # input_fre = self.model_fre(input_fre)

        # input_corp = torch.cat((input_time,input_fre),dim=1)
        input_corp = self.model_corp(input_time)
        # print(x.shape)
        
        return input_corp
    


In [ ]:
x_current = torch.zeros(80,16,7,7)
x_voltage = torch.zeros(80,1,7,7)
print(x_current.shape, x_voltage.shape)
fusing = torch.cat((x_current,x_voltage),dim=1)
print(fusing.shape)

In [ ]:
model = Model()
print(summary(model,input_size=[(1,1, 80, 80)]))


In [ ]:
import numpy as np
from scipy.fft import fft, ifft
from scipy import signal

# 生成示例信号（假设采样率为Fs）
Fs = 1000  # 采样率
T = 1/Fs  # 采样间隔
t = np.arange(0, 1, T)
signal_freq = 50  # 信号频率
signal_amplitude = 1
signal_noise = 0.2 * np.random.randn(len(t))  # 添加噪音
signal_data = signal_amplitude * np.sin(2 * np.pi * signal_freq * t) + signal_noise

# 进行FFT
signal_fft = fft(signal_data)

# 获取频率轴
freqs = np.fft.fftfreq(len(signal_data), T)
print(freqs)
# 识别主要频率分量
main_freq_index = np.argmax(np.abs(signal_fft))
main_freq = freqs[main_freq_index]
print(main_freq)
# 设计带阻滤波器
notch_freq = np.abs(main_freq)  # 带阻滤波器中心频率即为主要频率
Q = 10  # 带阻滤波器的质因数
b, a = signal.iirnotch(notch_freq, Q, Fs)

# 应用带阻滤波器
filtered_signal_fft = signal.lfilter(b, a, signal_fft)

# 逆FFT得到处理后的信号
filtered_signal = ifft(filtered_signal_fft)

# 可视化结果或进一步分析处理后的信号

In [3]:

"""
如果以6400个点做fft变换，因为采样率的原因，得到的频谱图的x轴的精度为10Hz，低于3000Hz成分只有300个点
"""

N09_M07_F10_K001_1 raw_data\K001\K001\N09_M07_F10_K001_1.mat
N09_M07_F10_KA04_1 raw_data\KA04\KA04\N09_M07_F10_KA04_1.mat


'\n如果以6400个点做fft变换，因为采样率的原因，得到的频谱图的x轴的精度为10Hz，低于3000Hz成分只有300个点\n'

In [7]:
def get_two_loader_from_1D_data(trainX,trainY,valX,valY):
    trainX,valX = torch.FloatTensor(trainX),torch.FloatTensor(valX)

    encoder = LabelEncoder()
    trainY = encoder.fit_transform(trainY.ravel())
    encoder = LabelEncoder()
    valY = encoder.fit_transform(valY.ravel())
    trainY,valY = torch.LongTensor(trainY),torch.LongTensor(valY)

    train_dataset =  torch.utils.data.TensorDataset(trainX, trainY)
    train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True, )
    test_dataset =  torch.utils.data.TensorDataset(valX, valY)
    test_loader = DataLoader(dataset=test_dataset, batch_size=len(valX), shuffle=True, )

    return train_loader,test_loader

In [ ]:
# 根据.pth创建模型实例
model_state_dict = torch.load('best_model_essay_optim.pth',map_location=torch.device('cpu'))
# 
model = Model()
# 设置模型为评估模式
model.load_state_dict(model_state_dict)
model.eval()# 将模型参数加载到模型实例中

In [14]:
import torch
q = torch.Tensor([1])
print(q.item())
q.item() == 1

1.0


True

In [17]:

def train_model_process(model,train_dataloader,val_dataloader,num_epoches,model_name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.99))
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    best_model_wts = copy.deepcopy(model.state_dict())

    best_acc = 0.0
    # train_loss_all = []
    # val_loss_all = []
    since = time.time()
    
    # with open(output_file, 'w') as f:

    for epoch in range(num_epoches):
        print("Epoch {}/{}".format(epoch + 1, num_epoches))
        print("-" * 10)

        # train_loss = 0.0
        # val_loss = 0.0
        # train_num = 0
        # val_num = 0
        y_true_all = None
        y_pred_all = None
        for step, (bx,by) in enumerate(train_dataloader):

            bx = bx.to(device)
            by = by.to(device)

            model.train()
            output = model(bx)

            # print(output)
            pre_lab = torch.argmax(output,dim = 1)
            loss = criterion(output, by)
            # print(pre_lab)
            if y_true_all == None:
                y_true_all = by
                y_pred_all = pre_lab
            else:
                y_true_all = torch.cat((y_true_all,by))
                y_pred_all = torch.cat((y_pred_all,pre_lab))
            

            
            # print('realy:',by)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # train_loss += loss.item()* bx.size(0)
            # train_num += bx.size(0)

        # train_loss_all.append(train_loss/train_num)

        evaluate_metrics(y_true_all.cpu(),y_pred_all.cpu(),model_name + "_train.csv",epoch+1)

        y_true_all = None
        y_pred_all = None
        for step, (bx,by) in enumerate(val_dataloader):

            bx = bx.to(device)
            by = by.to(device)

            model.eval()
            output = model(bx)

            pre_lab = torch.argmax(output,dim = 1)
            loss = criterion(output, by)

            if y_true_all == None:
                y_true_all = by
                y_pred_all = pre_lab
            else:
                y_true_all = torch.cat((y_true_all,by))
                y_pred_all = torch.cat((y_pred_all,pre_lab))

            # val_loss += loss.item() * bx.size(0)
            

            # val_num += bx.size(0)

        # train_loss_all.append(train_loss/train_num)
        test_acc = evaluate_metrics(y_true_all.cpu(),y_pred_all.cpu(),model_name + "_test.csv",epoch+1)


        if test_acc > best_acc:
            best_acc = test_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        if (epoch+1)%100 == 0:
            torch.save(best_model_wts, "pth/" + model_name + "_" + str(epoch+1) + ".pth")
            
        time_use = time.time() - since
        print("Time usage {:.0f}m{:.0f}s".format(time_use // 60, time_use % 60))


In [23]:

def valid_model(model,val_dataloader,model_name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.99))
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)

    model_state_dict = torch.load('pth/' + model_name + '.pth')

    # 设置模型为评估模式
    model.load_state_dict(model_state_dict)
   
    since = time.time()
    
    y_true_all = None
    y_pred_all = None
    for step, (bx,by) in enumerate(val_dataloader):

        bx = bx.to(device)
        by = by.to(device)

        model.eval()
        output = model(bx)

        # print(output)
        pre_lab = torch.argmax(output,dim = 1)
        loss = criterion(output, by)
        # print(pre_lab)
        if y_true_all == None:
            y_true_all = by
            y_pred_all = pre_lab
        else:
            y_true_all = torch.cat((y_true_all,by))
            y_pred_all = torch.cat((y_pred_all,pre_lab))
        

    evaluate_metrics(y_true_all.cpu(),y_pred_all.cpu(),model_name + "valid.csv",-1)


In [27]:
model_name = 'test_200'

model = Model()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
model = model.to(device)

model_state_dict = torch.load('pth/' + model_name + '.pth')

# 设置模型为评估模式
model.load_state_dict(model_state_dict)

def apply_model(model,data):
    
    since = time.time()

    model.eval()
    output = model(data)

    print(output)
    pre_lab = torch.argmax(output,dim = 1)
    print(pre_lab)

    return pre_lab




In [6]:
import pandas as pd
import numpy as np
groups = ["pump_100_BRB_H_filtered_cated\Electric_Motor-2_100_time-broken rotor bar-ch1.csv"]
filaname = "pump_100_BRB_H_filtered_cated\Electric_Motor-2_100_time-broken rotor bar-ch1.csv"
data = pd.read_csv(filaname)
data = data.values
print(data.shape)
tdata = data[:,0]
adata = data[:,1]
print(adata.shape)


(599579, 2)
(599579,)


In [8]:
import numpy as np
index = 0
highindex = len(adata)
print(highindex)
for i in range(100):
    since = time.time()
    
    if(index + 6400 >= highindex):
        sub = index + 6400 - highindex
        tem = adata[index:].copy() 
        tem = np.hstack((tem,adata[:sub].copy()))
        
        index = sub
    else:
        tem = adata[index:index+6400].copy()
        index += 6400
    # print(index, tem.shape)
    tem = (tem - min(tem)) / (max(tem) - min(tem)) * 255
    tem = torch.FloatTensor(tem.reshape(80,80))
    print(apply_model(model,tem))
    time_use = time.time() - since
# for i in range(0,len(adata)-6400,6400):
#     tem = adata[i:i+6400].copy()
#     tem = (tem - min(tem)) / (max(tem) - min(tem)) * 255
#     tem = torch.FloatTensor(tem.reshape(80,80))
    # normalized_data = (data_segment - min(data_segment)) / (max(data_segment) - min(data_segment)) * 255
    apply_model(model,tem)

599579
6400 (6400,)
12800 (6400,)
19200 (6400,)
25600 (6400,)
32000 (6400,)
38400 (6400,)
44800 (6400,)
51200 (6400,)
57600 (6400,)
64000 (6400,)
70400 (6400,)
76800 (6400,)
83200 (6400,)
89600 (6400,)
96000 (6400,)
102400 (6400,)
108800 (6400,)
115200 (6400,)
121600 (6400,)
128000 (6400,)
134400 (6400,)
140800 (6400,)
147200 (6400,)
153600 (6400,)
160000 (6400,)
166400 (6400,)
172800 (6400,)
179200 (6400,)
185600 (6400,)
192000 (6400,)
198400 (6400,)
204800 (6400,)
211200 (6400,)
217600 (6400,)
224000 (6400,)
230400 (6400,)
236800 (6400,)
243200 (6400,)
249600 (6400,)
256000 (6400,)
262400 (6400,)
268800 (6400,)
275200 (6400,)
281600 (6400,)
288000 (6400,)
294400 (6400,)
300800 (6400,)
307200 (6400,)
313600 (6400,)
320000 (6400,)
326400 (6400,)
332800 (6400,)
339200 (6400,)
345600 (6400,)
352000 (6400,)
358400 (6400,)
364800 (6400,)
371200 (6400,)
377600 (6400,)
384000 (6400,)
390400 (6400,)
396800 (6400,)
403200 (6400,)
409600 (6400,)
416000 (6400,)
422400 (6400,)
428800 (6400,)
4352

In [9]:
dicts = {'Electric_Motor-2_50_time-broken rotor bar-ch3.csv':1,'Electric_Motor-2_50_time-healthy 1-ch2.csv':0,
        'Electric_Motor-2_75_time-broken rotor bar-ch3':1,'Electric_Motor-2_75_time-healthy 1-ch2':0,
        'Electric_Motor-2_100_time-broken rotor bar-ch2':1,'Electric_Motor-2_100_time-healthy 1-ch2':0}
train_x, train_y, val_x, val_y = load_data_and_convert_to_grid(dicts,0.9,'N20khz')

Getting data with keyword Electric_Motor-2_50_time-broken rotor bar-ch3.csv
Getting data with keyword Electric_Motor-2_50_time-healthy 1-ch2.csv
Getting data with keyword Electric_Motor-2_75_time-broken rotor bar-ch3
Getting data with keyword Electric_Motor-2_75_time-healthy 1-ch2
Getting data with keyword Electric_Motor-2_100_time-broken rotor bar-ch2
Getting data with keyword Electric_Motor-2_100_time-healthy 1-ch2
(246,) (246, 6400)
(30,) (30, 6400)


In [6]:
model = Model()

datadicts = {'K001':0,'KA04':1}
# for each in datadicts:
#     get_data_and_convert_to_pixels(each,datadicts[each],6400,6400,128000,output_dir="G_lp_td_fd_pixel")
data_foldername= "G_lp_td_fd_pixel"
sample_length = 8901
trainX,trainY,valX,valY = load_data_and_convert_to_grid(datadicts,0.92,data_foldername,sample_length)


N09_M07_F10_K001_1 raw_data\K001\K001\N09_M07_F10_K001_1.mat
N09_M07_F10_K001_10 raw_data\K001\K001\N09_M07_F10_K001_10.mat
N09_M07_F10_K001_11 raw_data\K001\K001\N09_M07_F10_K001_11.mat
N09_M07_F10_K001_12 raw_data\K001\K001\N09_M07_F10_K001_12.mat
N09_M07_F10_K001_13 raw_data\K001\K001\N09_M07_F10_K001_13.mat
N09_M07_F10_K001_14 raw_data\K001\K001\N09_M07_F10_K001_14.mat
N09_M07_F10_K001_15 raw_data\K001\K001\N09_M07_F10_K001_15.mat
N09_M07_F10_K001_16 raw_data\K001\K001\N09_M07_F10_K001_16.mat
N09_M07_F10_K001_17 raw_data\K001\K001\N09_M07_F10_K001_17.mat
N09_M07_F10_K001_18 raw_data\K001\K001\N09_M07_F10_K001_18.mat
N09_M07_F10_K001_19 raw_data\K001\K001\N09_M07_F10_K001_19.mat
N09_M07_F10_K001_2 raw_data\K001\K001\N09_M07_F10_K001_2.mat
N09_M07_F10_K001_20 raw_data\K001\K001\N09_M07_F10_K001_20.mat
N09_M07_F10_K001_3 raw_data\K001\K001\N09_M07_F10_K001_3.mat
N09_M07_F10_K001_4 raw_data\K001\K001\N09_M07_F10_K001_4.mat
N09_M07_F10_K001_5 raw_data\K001\K001\N09_M07_F10_K001_5.mat
N0

In [11]:
train_loader,test_loader = get_two_loader_from_1D_data(train_x, train_y, val_x, val_y)

In [26]:
model2 = Model()
pth_name = 'test_200'
valid_model(model2,test_loader,pth_name)

[Info] acc: 1.0, precision: 1.0, recall: 1.0, f1: 1.0
[[15  0]
 [ 0 15]]


In [19]:
model = Model()
model_name = 'test'
train_model_process(model,train_loader,test_loader,200,model_name)

Epoch 1/200
----------


d:\conda\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\conda\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[Info] acc: 0.5040650406504065, precision: 0.6697478991596638, recall: 0.5040650406504065, f1: 0.5603576751117735
[[88 24 11]
 [31 36 56]
 [ 0  0  0]]
[Info] acc: 0.16666666666666666, precision: 0.22727272727272727, recall: 0.16666666666666666, f1: 0.19230769230769226
[[0 6 9]
 [1 5 9]
 [0 0 0]]
Time usage 0m0s
Epoch 2/200
----------


d:\conda\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\conda\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[Info] acc: 0.5975609756097561, precision: 0.6972500511142916, recall: 0.5975609756097561, f1: 0.6206613187243201
[[99 19  5]
 [47 48 28]
 [ 0  0  0]]
[Info] acc: 0.5, precision: 0.25, recall: 0.5, f1: 0.3333333333333333
[[ 0 15]
 [ 0 15]]
Time usage 0m1s
Epoch 3/200
----------


...

[Info] acc: 0.8333333333333334, precision: 0.8414815817984833, recall: 0.8333333333333334, f1: 0.8323331393899094
[[ 93  30]
 [ 11 112]]
[Info] acc: 0.5, precision: 0.25, recall: 0.5, f1: 0.3333333333333333
[[ 0 15]
 [ 0 15]]
Time usage 0m5s
Epoch 14/200
----------


d:\conda\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[Info] acc: 0.8455284552845529, precision: 0.8549120782130492, recall: 0.8455284552845529, f1: 0.8445006321112517
[[ 94  29]
 [  9 114]]
[Info] acc: 0.5, precision: 0.25, recall: 0.5, f1: 0.3333333333333333
[[ 0 15]
 [ 0 15]]
Time usage 0m5s
Epoch 15/200
----------


d:\conda\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[Info] acc: 0.8536585365853658, precision: 0.8613981762917933, recall: 0.8536585365853658, f1: 0.8528708133971292
[[ 96  27]
 [  9 114]]
[Info] acc: 0.6, precision: 0.7777777777777779, recall: 0.6, f1: 0.5238095238095238
[[ 3 12]
 [ 0 15]]
Time usage 0m5s
Epoch 16/200
----------
[Info] acc: 0.8658536585365854, precision: 0.8729784366576819, recall: 0.8658536585365854, f1: 0.8652099556677237
[[ 98  25]
 [  8 115]]
[Info] acc: 0.6666666666666666, precision: 0.8, recall: 0.6666666666666666, f1: 0.625
[[ 5 10]
 [ 0 15]]
Time usage 0m6s
Epoch 17/200
----------
[Info] acc: 0.8699186991869918, precision: 0.872380065207266, recall: 0.8699186991869918, f1: 0.8697033898305083
[[102  21]
 [ 11 112]]
[Info] acc: 0.6666666666666666, precision: 0.8, recall: 0.6666666666666666, f1: 0.625
[[ 5 10]
 [ 0 15]]
Time usage 0m6s
Epoch 18/200
----------
[Info] acc: 0.8699186991869918, precision: 0.8703103288559519, recall: 0.8699186991869918, f1: 0.8698842975206612
[[105  18]
 [ 14 109]]
[Info] acc: 0.833333